#  Évaluation de LLMs sur la tâche de résumé automatique


## Partie I : Installation et import des bibliothèques

In [17]:
!pip install -q rouge_score==0.1.2 evaluate datasets nltk transformers --upgrade
import nltk
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
from datasets import load_dataset
import pandas as pd

# Chargement du dataset CNN/DailyMail
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_dataset = dataset["train"].select(range(100))  # échantillon de 100
test_dataset = dataset["test"].select(range(50))     # échantillon de 50

# Conversion en DataFrame pour exploration
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)

# Affichage d'un exemple
train_df[['article', 'highlights']].head(1)


,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...


##  Partie II : Exploration des données

In [20]:
# Aperçu général du dataset d'entraînement
print(f"Nombre d'exemples : {len(train_df)}")
train_df[['article', 'highlights']].sample(3)

Nombre d'exemples : 100


,article,highlights
87,"MADRID, Spain -- Lionel Messi scored for the s...",Lionel Messi scores for the sixth game in a ro...
49,(Real Simple) -- Here are five great ways to e...,Real Simple tips can add up to great summer .\...
80,"HELSINKI, Finland (CNN) -- An 18-year-old auth...","NEW: Teen gunman is dead, Finnish police say ...."


##  Partie III : Génération de résumés avec T5-small

In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch, gc

def summarize_with_t5(texts, model_name="t5-small"):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    summaries = []
    for text in texts:
        input_text = "summarize: " + text
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
        output = model.generate(**inputs, max_new_tokens=80)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)

        torch.cuda.empty_cache()
        gc.collect()

    return summaries

# Application sur 10 articles
sample_articles = train_df["article"].head(10).tolist()
t5_summaries = summarize_with_t5(sample_articles)


##  Partie IV : Affichage des résumés générés vs attendus

In [22]:
comparison_df = pd.DataFrame({
    "Article": sample_articles,
    "Résumé Référence": train_df["highlights"].head(10).tolist(),
    "Résumé T5": t5_summaries
})
comparison_df[['Résumé Référence', 'Résumé T5']].head()

,Résumé Référence,Résumé T5
0,Harry Potter star Daniel Radcliffe gets £20M f...,he says he has no plans to fritter cash away o...
1,Mentally ill inmates in Miami are housed on th...,inmates with most severe mental illnesses are ...
2,"NEW: ""I thought I was going to die,"" driver sa...","driver on the bridge says he had a 30-, 35-foo..."
3,"Five small polyps found during procedure; ""non...",new: president's doctor recommended repeat pro...
4,"NEW: NFL chief, Atlanta Falcons owner critical...",new: nfl suspends quarterback without pay. new...


##  Partie V : Évaluation avec la métrique ROUGE

In [23]:
import evaluate
from nltk.tokenize import sent_tokenize

rouge = evaluate.load("rouge")

def compute_rouge(preds, refs):
    preds = ["\n".join(sent_tokenize(p)) for p in preds]
    refs = ["\n".join(sent_tokenize(r)) for r in refs]
    results = rouge.compute(predictions=preds, references=refs)
    return results

# Assurez-vous que t5_summaries contient bien 10 résumés générés
rouge_scores = compute_rouge(t5_summaries, train_df["highlights"].head(10).tolist())
print(rouge_scores)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


##  Conclusion métier
L’objectif de ce projet était d’évaluer les performances d’un LLM sur une tâche de résumé. Nous avons pu constater que la génération automatique de résumés par un modèle comme T5-small est globalement correcte, mais dépend fortement du modèle utilisé et du type de texte. Les scores ROUGE nous offrent une mesure objective mais partielle de qualité, notamment sur la couverture et la fidélité du résumé. Ce type d’évaluation est essentiel pour affiner les choix de modèle dans des applications de production (médias, recherche, veille, etc.).